In [29]:

import pandas as pd
import matplotlib as plt
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp


In [30]:
# Evitamos que los números aparezcan en notación científica:
pd.set_option("display.float_format", lambda x: "{:,.1f}".format(x))

In [31]:
# Cambio el límite de filas mostradas:
pd.set_option("display.max_rows", 15)

In [32]:
# Obtención del dataframe con datos de población globales.
path_global_population_csv = "./Data/population.csv"
df_global_population = pd.read_csv(path_global_population_csv)


In [33]:
# Limpieza del dataframe para quedarnos únicamente con los valores para China y las columnas de año y población total.
df_china_population = df_global_population.copy()
df_china_population = df_china_population.loc[df_china_population["Country name"] == "China"].reset_index()
df_china_population = df_china_population[["Year", "Population"]]
df_china_population = df_china_population.rename(columns={"Year": "Año", "Population" : "Población"})
df_china_population

,Año,Población
0,1950,543979200
1,1951,553614000
2,1952,564954500
3,1953,577378700
4,1954,589936000
...,...,...
67,2017,1410276000
68,2018,1417069400
69,2019,1421864100
70,2020,1424929800


In [42]:
# Ahora este DF está listo para ser comprobado visualmente.
fig_china_population = px.line(df_china_population, x= "Año", y= "Población",
                               title = "Evolución de la población en China",
                               color_discrete_sequence=["red"])
fig_china_population.update_yaxes(tickformat="0,.0f", ticksuffix="M") # Cambio el formato de los valores en el eje Y.
fig_china_population.update_layout(template="plotly_dark") 

fig_china_population.show()

# Al aportar una idea genérica es útil como introducción al tema del EDA.

In [35]:
# Ahora queremos comprobar la tasa de variación interanual.
df_var_interanual_china = df_china_population.copy()
df_var_interanual_china["Crecimiento interanual"] = df_var_interanual_china["Población"].pct_change(periods = 1)*100

# Gestiono el valor NaN de la primera fila (ya que no se puede calcular variación). En este caso pongo el mismo que la fila siguiente.
df_var_interanual_china["Crecimiento interanual"].fillna(method="bfill", inplace=True)
df_var_interanual_china

,Año,Población,Crecimiento interanual
0,1950,543979200,1.8
1,1951,553614000,1.8
2,1952,564954500,2.0
3,1953,577378700,2.2
4,1954,589936000,2.2
...,...,...,...
67,2017,1410276000,0.6
68,2018,1417069400,0.5
69,2019,1421864100,0.3
70,2020,1424929800,0.2


In [44]:
# Generando un gráfico de líneas con la información de la variación interanual puedo darme cuenta de anomalías y cambios en la tendencia:

fig_var_interanual_china = px.line(df_var_interanual_china, x= "Año", y= "Crecimiento interanual",
                                    title = "Crecimiento interanual de la población en China (en %)",
                                    color_discrete_sequence=["red"])
fig_var_interanual_china.update_layout(template="plotly_dark") 
fig_var_interanual_china.show()

# Vemos que hay una caída brusca entre 1958 y 1961 que luego remonta.
# Además, a partir de los años 60 la tendencia es a la baja.
# Hay explicación para estos dos fenómenos.

In [37]:
# Puede ser interesante compararlo con la media mundial para el mismo período, así que sacaremos esos datos:
df_global_mean = df_global_population.copy()
df_global_mean = df_global_mean[["Country name", "Year", "Population"]]
df_global_mean = df_global_mean.rename(columns={"Country name": "País", "Year": "Año", "Population" : "Población"})
df_global_mean["Crecimiento interanual"] = df_global_mean["Población"].pct_change(periods = 1)*100

df_global_mean = df_global_mean.drop("País", axis = 1).groupby("Año")["Crecimiento interanual"].mean().reset_index()
df_global_mean = df_global_mean.rename(columns={"Crecimiento interanual" : "Crecimiento interanual medio"})

# Como me da un valor extraño para la primera fila, la relleno con el falor de la siguiente fila.
df_global_mean["Crecimiento interanual medio"][0] = df_global_mean["Crecimiento interanual medio"][1]




df_global_mean




C:\Users\prado\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Año,Crecimiento interanual medio
0,1950,2.0
1,1951,2.0
2,1952,2.0
3,1953,2.0
4,1954,2.1
...,...,...
67,2017,1.1
68,2018,1.2
69,2019,1.2
70,2020,1.0


In [45]:
# Creo una figura para comprobar la tendencia media global.

fig_global_variation_mean = px.line(df_global_mean, x= "Año", y= "Crecimiento interanual medio",
                                    title = "Crecimiento medio de la población mundial (en %)")
fig_global_variation_mean.update_layout(template="plotly_dark") 

fig_global_variation_mean.show()

In [60]:
# Creo una figura superponiendo ambas variables para comprobar si se asemejan o difieren.

fig_comparar_var_pobl = px.line(df_var_interanual_china, x="Año", y="Crecimiento interanual",
                                color_discrete_sequence=["red"],
                                title="Crecimiento anual de la población china vs mundial (en %)")
fig_comparar_var_pobl.add_trace(fig_global_variation_mean.data[0])
fig_comparar_var_pobl.update_layout(template="plotly_dark")

fig_comparar_var_pobl.show()


In [61]:
# Hago una copia de la figura anterior añadiendo una anotación explicativa.

fig_gran_salto_adelante = go.Figure(fig_comparar_var_pobl)
fig_gran_salto_adelante.add_annotation(x=1958, y=df_var_interanual_china.loc[df_var_interanual_china["Año"] == 1958, "Crecimiento interanual"].iloc[0],
                    text="Gran Salto Adelante",
                    showarrow=True, arrowhead=5)
fig_gran_salto_adelante.show()

In [64]:
# Creo otra copia a partir de la figura anterior para añadir otra anotación relevante más.
fig_control_natalidad = go.Figure(fig_gran_salto_adelante)
fig_control_natalidad.add_annotation(x=1980, y=df_var_interanual_china.loc[df_var_interanual_china["Año"] == 1980, "Crecimiento interanual"].iloc[0],
                    text="Hijo único",
                    showarrow=True, arrowhead=5)
fig_control_natalidad.show()